<a href="https://colab.research.google.com/github/Cypher29/AI_repo/blob/main/OfficeAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Install required packages**

In [ ]:
!pip install -q transformers datasets diffusers

In [ ]:
pip install --upgrade gradio

In [ ]:
pip install DateTime

Import **packages**

In [ ]:
from google.colab import userdata
import gradio as gr
from openai import OpenAI
from datetime import date
import json
import os
import pdb

**Initiate connection to OpenAI**

In [ ]:
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_KEY')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

System Prompt

In [ ]:
system_message = "You are a helpful assistant for a software company called 'GamingDesign Ltd'"
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [ ]:
EmployeeRecords = [ {"ID": "1011", "Name": "John","Designation":"Manager","location": "Hyderabad"},
 {"ID": "1015", "Name": "Carles","Designation":"Sr Developer","location":"Bengaluru"},
  {"ID": "1016", "Name": "Rock","Designation":"Developer","location":"Hyderabad"}]

def get_employee_details(employee_name):
    #pdb.set_trace()
    print(f"Tool get_employe_details called for {employee_name}")
    employee_name = employee_name.lower()
    for employee in EmployeeRecords:
        if employee_name.lower() in employee.get("Name").lower():
            return employee
    return []

In [ ]:
get_employee_details('John')

In [ ]:

MeetingRoomRecords = [{
    "roomName"  : "Newton",
    "location"  : "Hyderabad",
},
  {
    "roomName"  : "Einstein",
    "location"  : "Hyderabad",
},
  {
    "roomName"  : "Raman",
    "location"  : "Chennai",
},]

def get_meetingroom_details(meetingroom_name, meetingroom_location ):
    meetingrooms= []
    if meetingroom_name and meetingroom_location:
        for meetingroom in MeetingRoomRecords:
            if meetingroom.get("location").lower() == meetingroom_location.lower() and meetingroom.get("roomName").lower() == meetingroom_name.lower():
                return True
        return False
    elif meetingroom_name:
        for meetingroom in MeetingRoomRecords:
            if meetingroom.get("roomName").lower() == meetingroom_name.lower():
              return meetingroom.get("location")
    elif meetingroom_location:
        for meetingroom in MeetingRoomRecords:
            if meetingroom.get("location").lower() == meetingroom_location.lower():
                meetingrooms.append(meetingroom)
        return meetingrooms
    else:
        for meetingroom in MeetingRoomRecords:
            meetingrooms.append(meetingroom.get("roomName"))
        return meetingrooms


    return  "unknown"

In [ ]:
def get_todays_date():
    return str(date.today())

In [ ]:
import datetime
from datetime import date

booking_information = {
    "Newton":[
        {"from": "05/15/25 10:30:00", "to": "05/05/25 11:30:00"},
        {"from": "05/23/25 14:00:00", "to": "05/23/25 15:00:00"},
        {"from": "05/25/25 11:00:00", "to": "05/25/25 11:30:00"},
        {"from": "05/25/25 13:00:00", "to": "05/25/25 14:00:00"}
    ],
    "Raman":[
        {"from": "05/10/25 09:15:00", "to": "05/10/25 10:00:00"},
        {"from": "05/11/25 15:00:00", "to": "05/11/25 16:00:00"}
    ],
    "Einstein":[
        {"from": "05/06/25 15:00:00", "to": "05/06/25 16:30:00"},
        {"from": "05/10/25 10:15:00", "to": "05/10/25 10:45:00"},
        {"from": "05/11/25 12:00:00", "to": "05/11/25 15:00:00"}
    ]

}

def get_meetingroom_bookings(meetingroom_name, meetingdate, fromtime, totime ):
    if not meetingroom_name:
        return "unknown"
   # pdb.set_trace()
    for key, value in booking_information.items():
        if key.lower() in meetingroom_name.lower():
            for booking in value:
                from_datetime = datetime.datetime.strptime(meetingdate+" "+fromtime, '%m/%d/%y %H:%M:%S')
                to_datetime = datetime.datetime.strptime(meetingdate+" "+totime, '%m/%d/%y %H:%M:%S')
                meeting_from_datetime = datetime.datetime.strptime(booking.get("from"),'%m/%d/%y %H:%M:%S')
                meeting_to_datetime = datetime.datetime.strptime(booking.get("to"),'%m/%d/%y %H:%M:%S')
                if meeting_from_datetime <= from_datetime and   meeting_to_datetime > from_datetime or meeting_from_datetime <= to_datetime and  meeting_to_datetime > to_datetime:
                    return 1
                if(from_datetime <= meeting_from_datetime and to_datetime >= meeting_to_datetime):
                    return 1
            return 2
    return 0


In [ ]:
get_meetingroom_bookings("Einstein","06/05/25","03:00:00","04:30:00")

In [ ]:
get_meetingroom_details("Raman",None)

In [ ]:
employee_details_function = {
    "name": "get_employee_details",
    "description": "Get the details of an employee, like designation, location and id. Call this whenever you need to know the employee details or employee information, for example when a customer asks 'give details about this employee' or ' who is this employee' or 'what is id of this employee' or 'location of the employee'",
    "parameters": {
        "type": "object",
        "properties": {
            "employee_name": {
                "type": "string",
                "description": "Name of the employee whose details wanted",
            },
        },
        "required": ["employee_name"],
        "additionalProperties": False
    }
}

In [ ]:
tools = [{"type": "function", "function": employee_details_function}]

In [ ]:
meetingroom_details_functions ={
    "name": "get_meetingroom_details",
    "description": "Get the details of a meeting room, like meeting room names and location, for example when a customer asks 'where is this meeting room ' or ' gice rooms in this location'",
    "parameters": {
        "type": "object",
        "properties": {
            "meetingroom_name": {
                "type": "string",
                "description": "Name of the meetingroom",
            },
            "meetingroom_location": {
                "type": "string",
                "description": "Name of the meetingroom",
            },
        },
        "additionalProperties": False
  }
}


In [ ]:
meetingroom_bookings_functions ={
    "name": "get_meetingroom_bookings",
    "description": "check the meeting room availability, convert given time format as HH:MM:SS and date format as MM:DD:YY, this checks meeting room is free on perticular date time, for example when a customer asks 'is the meeting room available between time ' or 'check meeting room availability'",
    "parameters": {
        "type": "object",
        "properties": {
            "meetingroom_name": {
                "type": "string",
                "description": "Name of the meetingroom",
            },
            "meetingdate": {
                "type": "string",
                "description": "Date of the meeting",
            },
            "fromtime": {
                "type": "string",
                "description": "start time of the meeting",
            },
            "totime": {
                "type": "string",
                "description": "end time of the meeting"
            },
        },
        "required": ["meetingroom_name","meetingdate","fromtime","totime"],
        "additionalProperties": False
  }
}


In [ ]:
todays_date_functions ={
    "name": "get_todays_date",
    "description": "Get the today's date, for example when a customer asks 'what is today's date ' or 'today's date'",
    "additionalProperties": False
  }


In [ ]:
tools = [{"type": "function", "function": employee_details_function},
         {"type": "function", "function": meetingroom_details_functions},
         {"type": "function", "function": todays_date_functions},
         {"type": "function", "function": meetingroom_bookings_functions}]

In [ ]:
#import pdb
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    #pdb.set_trace()
    #messages = [{"role": "system", "content": system_message}] + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools, tool_choice="auto")


    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools, tool_choice="auto")
        if response.choices[0].finish_reason=="tool_calls":
              message = response.choices[0].message
              response = handle_tool_call(message)
              messages.append(message)
              messages.append(response)
              response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools, tool_choice="auto")
              if response.choices[0].finish_reason=="tool_calls":
                  message = response.choices[0].message
                  response = handle_tool_call(message)
                  messages.append(message)
                  messages.append(response)
                  response = openai.chat.completions.create(model=MODEL, messages=messages)
    #pdb.set_trace()
  #  talker(response.choices[0].message.content)
   # audio_stream = BytesIO(response.choices[0].message.content)
   # audio = AudioSegment.from_file(audio_stream, format="mp3")
   # play(audio)
    return response.choices[0].message.content

In [ ]:
#chat("Meeting room name Newton and time between '09:00:00' and '10:20:00'",[])
#chat("check meeting room Newton availability on 01/05/25 between '09:00:00' and '10:20:00'",[])
#chat("check meeting room Newton availability today between '09:00:00' and '10:20:00'",[])
#chat("availability of meeting room today at 9 am to 10 am",[])
chat("check Einstein room between 3 to 5:30 pm on today",[])
#chat("Can you check Newton is free today",[])
#chat("Who are you",[])

In [ ]:

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    #pdb.set_trace()
    if tool_call.function.name == "get_todays_date":
        todayDate = get_todays_date()
        response = {
              "role": "tool",
              "content": json.dumps({"Today's date": todayDate}),
              "tool_call_id": tool_call.id
            }
    elif tool_call.function.name == "get_employee_details":
        employee_name = arguments.get('employee_name')
        employeeRecord = get_employee_details(employee_name)
        if employeeRecord:
            response = {
              "role": "tool",
              "content": json.dumps({"employee_name": employee_name,"emploee ID": employeeRecord.get("ID"),"Employee Designation": employeeRecord.get("Designation"),"Employee location": employeeRecord.get("location")}),
              "tool_call_id": tool_call.id
            }
        else:
            response = {
            "role": "tool",
            "content": json.dumps({"employee_name": employee_name,"error": "Employee not found"}),
            "tool_call_id": tool_call.id
        }
    elif tool_call.function.name == "get_meetingroom_details":
        meetingroom_name = arguments.get('meetingroom_name')
        if not meetingroom_name:
            meetingroom_name = None
        meetingroom_location = arguments.get('meetingroom_location')
        if not meetingroom_location:
            meetingroom_location = None
        result = get_meetingroom_details(meetingroom_name,meetingroom_location)
        if type(result) == bool:
          if result == True:
            response = {
              "role": "tool",
              "content": json.dumps({"meetingroom_name":meetingroom_name, "meetingroom_location":meetingroom_location, "info": "meeting room is exist in the current location"}),
              "tool_call_id": tool_call.id
            }
          else:
            response = {
              "role": "tool",
              "content": json.dumps({"meetingroom_name":meetingroom_name, "meetingroom_location":meetingroom_location, "info": "meeting room is not exist in the current location"}),
              "tool_call_id": tool_call.id
            }
        else:
          if meetingroom_name:
            response = {
              "role": "tool",
              "content": json.dumps({"meetingroom_name":meetingroom_name, "meetingroom_location":result, "info": "meeting room is in this location"}),
              "tool_call_id": tool_call.id
            }
          else:
            response = {
              "role": "tool",
              "content": json.dumps({"meetingroom_name":result, "meetingroom_location":meetingroom_location, "info": "these meeting rooms are exist in the current location"}),
              "tool_call_id": tool_call.id
            }
    elif tool_call.function.name == "get_meetingroom_bookings":
        meetingroom_name = arguments.get('meetingroom_name')
        meetingdate = arguments.get('meetingdate')
        fromtime = arguments.get('fromtime')
        totime = arguments.get('totime')
        result = get_meetingroom_bookings(meetingroom_name,meetingdate,fromtime,totime)
        if result == 0:
          response = {
              "role": "tool",
              "content": json.dumps({"meetingroom_name":meetingroom_name, "meetingdate":meetingdate, "fromtime":fromtime,"totime":totime,"error": "meeting room doesn not exist in this company, please ask for existing meeting rooms"}),
              "tool_call_id": tool_call.id
            }
        elif result == 1:
          response = {
              "role": "tool",
              "content": json.dumps({"meetingroom_name":meetingroom_name, "meetingdate":meetingdate, "fromtime":fromtime,"totime":totime,"info": "meeting room is not available it is booked by some one"}),
              "tool_call_id": tool_call.id
            }
        else:
          response = {
              "role": "tool",
              "content": json.dumps({"meetingroom_name":meetingroom_name, "meetingdate":meetingdate, "fromtime":fromtime,"totime":totime,"info": "meeting room is available to book"}),
              "tool_call_id": tool_call.id
            }
    else:
        response = {
            "role": "tool",
            "content": json.dumps({"employee_name": employee_name,"error": "I don't have this knowledge"}),
            "tool_call_id": tool_call.id
        }
    return response

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch(debug=True)

**Working with Audio AI models **

In [ ]:
!pip install -q transformers datasets diffusers


!ffmpeg -version
!ffprobe -version
!ffplay -version



!pip install simpleaudio



In [ ]:
import torch
from google.colab import userdata
from huggingface_hub import login
from transformers import pipeline
from datasets import load_dataset


from pydub import AudioSegment
from pydub.playback import play
from io import BytesIO
from IPython.display import Audio

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
hf_token = userdata.get('HUGGINGFACE_KEY')
login(hf_token, add_to_git_credential=True)

In [ ]:
synthesiser = pipeline("text-to-speech", "microsoft/speecht5_tts", device='cuda')

embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
speaker_embedding = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

speech = synthesiser("Hi to an artificial intelligence engineer, on the way to mastery!", forward_params={"speaker_embeddings": speaker_embedding})

sf.write("speech.wav", speech["audio"], samplerate=speech["sampling_rate"])
Audio("speech.wav")

In [ ]:
import simpleaudio as sa
def talker(message):
  synthesiser = pipeline("text-to-speech", "microsoft/speecht5_tts", device='cuda')

  embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
  speaker_embedding = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

  speech = synthesiser(message, forward_params={"speaker_embeddings": speaker_embedding})

  sf.write("speech.wav", speech["audio"], samplerate=speech["sampling_rate"])

  Audio("speech.wav", autoplay=True)


In [ ]:
import simpleaudio as sa
def talker(message):
  synthesiser = pipeline("text-to-speech", "microsoft/speecht5_tts", device='cuda')

  embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
  speaker_embedding = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

  speech = synthesiser(message, forward_params={"speaker_embeddings": speaker_embedding})

  sf.write("speech.wav", speech["audio"], samplerate=speech["sampling_rate"])

# Load and play audio using simpleaudio
  wave_obj = sa.WaveObject.from_wave_file("speech.wav")
  play_obj = wave_obj.play()
  play_obj.wait_done()  # Wait for playback to finish

    # Clean up the temporary file afterward
  os.remove("speech.wav")


In [ ]:
talker("Well, hi there")